## Practical 4. Constrained Optimization: Equality Constraints

#### Nils Lennier Mattiß, Federica Valeau, Eduard Mihai


In [11]:
import numpy as np
import math

In [12]:
def step(x, lamb, gradf, hessianf, h, gradh, hessianh, stepsize=1):
    """
    all functions should return numpy arrays of a proper size
    :param x: np.array of size
    :param lamb: np.array of size number of constraints (nc) x 1
    :param gradf: callable returning np.array of size number of arguments (na) x 1
    :param hessianf: callable returning np.array of size na x na
    :param h: callable returning np.array of size nc x 1
    :param gradh: np.array of size callable na x nc
    :param hessianh: np.array of size callable
    :param stepsize: optinial float for the stepsize
    :return: x: one step towards optimal solution of quadratic approximation
    """

    hessianLagrangian = hessianf(x) - lamb*hessianh(x)
    gradh = gradh(x).reshape((gradh(x).shape[0],1))

    topPart = np.concatenate((hessianLagrangian, -gradh), axis=1)
    bottomPart = np.concatenate((-gradh.T, np.zeros((gradh.shape[1], gradh.shape[1]))), axis=1)

    mat = np.concatenate((topPart, bottomPart))
    rhs = np.concatenate((-(gradf(x)-lamb*gradh.T), h(x)), axis=1).reshape((3,1))
    try:
        res = np.linalg.solve(mat, rhs)

        xdir = res[0:len(res)-1].reshape(x.shape)
        lambdir = res[-1]
        return x + xdir, lamb + lambdir
    except:
        return x, lamb



The functions and their derivatives used in the example

In [13]:
def gradf(x):
    return np.array([3*math.exp(3*x[0]), -4*math.exp(-4*x[1])])

def gradh(x):
    return np.array([2*x[0],2*x[1]])

def h(x):
    return np.array([x[0]**2 + x[1]**2 - 1]).reshape((1,1))

def hessianh(x):
    return np.array([2,0,0,2]).reshape(2,2)

def hessianf(x):
    return np.array([9*math.exp(3*x[0]),0,0,16*math.exp(-4*x[1])]).reshape((2,2))


In [14]:
# Try our Step function with the given example

x, l = step(np.array([-1, 1]), -1, gradf, hessianf, h, gradh, hessianh)
print(x,l)

[-0.77422564  0.72577436] [-0.35103786]


Here we show that the step function works and that it gives correctly the next set of values to be considered

##### Newton Method

In [15]:
def optimise(x_start, lamb_start, gradf, hessianf, h, gradh, hessianh, steps=100, stepsize=1):
    x, lamb = x_start, lamb_start
    for i in range(steps):
        #some error handling to get running code
        try:
            x, lamb = step(x, lamb, gradf, hessianf, h, gradh, hessianh)
        except OverflowError:
            break
    return x, lamb

#### Ex1

In [16]:
x = np.array([50, 100])
lamb = 5
steps = 100
print(optimise(x, lamb, gradf, hessianf, h, gradh, hessianh, steps))


(array([-0.74833549,  0.66332043]), array([-0.21232494]))


We can see that after 100 steps, by using the starting point given in the example our newton method converges to the solution. $(-0.74833549,  0.66332043)$ it's the solution that our algorithm converged to, which is the correct one, with $\lambda$ being equal -0.21232494

#### Ex2
Furthermore, we will test our solution on more examples to see how it performs. By checking the results of the experiments with the known results we can determine how close the approximation is.


In [17]:
#some intuitively chosen startpoints
intuitiveX = [[0,0], [0.05,0.05], [-1,1], [-10,10], [-100, 100], [1,-1], [10,-10], [100, -100], [1,1], [1.8, 1.8], [1.85, 1.85], [5,5], [-1,-1], [-1.8, -1.8], [-1.85, -1.85], [-5, -5]]
randomX = np.random.uniform(-100, 100, (20,2))          #some random points
startX = np.append(randomX, intuitiveX, axis=0)
startL = [-10, -5, -1, -0.1, 0, 0.1, 1, 5, 10]
startingPoints = [[-np.array(x), l] for x in startX for l in startL]
steps = 500
x_optimal = np.array([-0.74834, 0.66332])
lamb_optimal = -0.21233


for x,lamb in startingPoints:
    reached = "     failed     "
    x_reach, lamb_reach = optimise(x, lamb, gradf, hessianf, h, gradh, hessianh, steps)
    if np.linalg.norm(x_reach-x_optimal) < 1e-5:
        reached = "'''successful'''"
        if np.linalg.norm(lamb_reach-lamb_optimal) < 1e-4:
            reached = "|||super successful|||"
    print(f"{reached}.  after {steps} iterations we got from {x} with initial lambda {lamb} to {x_reach, lamb_reach}")


     failed     .  after 500 iterations we got from [ -0.90927292 -96.70620254] with initial lambda -10 to (array([5115.02628808,  -96.45620254]), array([-57919.01526512]))
     failed     .  after 500 iterations we got from [ -0.90927292 -96.70620254] with initial lambda -5 to (array([5115.02628808,  -96.45620254]), array([-29787.00190342]))
     failed     .  after 500 iterations we got from [ -0.90927292 -96.70620254] with initial lambda -1 to (array([5115.02628808,  -96.45620254]), array([-7281.39121406]))
     failed     .  after 500 iterations we got from [ -0.90927292 -96.70620254] with initial lambda -0.1 to (array([5115.02628808,  -96.45620254]), array([-2217.62880895]))
     failed     .  after 500 iterations we got from [ -0.90927292 -96.70620254] with initial lambda 0 to (array([5115.02628808,  -96.45620254]), array([-1654.98854172]))
     failed     .  after 500 iterations we got from [ -0.90927292 -96.70620254] with initial lambda 0.1 to (array([5115.02628808,  -96.456202

What we can observe is, that with starting points,r that have the same signs as the optimal point the newton like approach has no problem to converge to the optimal solution.
If the signs are changed the behaviour changes drastically.
For starting points with the same sign it seems like we converge until somewhere in the interval $\pm [0.5, 1.85]$
The choice of the initial lambda seems to only matter in the border region of the convergence area, but we can't really determine a pattern in that.
To conclude: Minus Plus is handled well, Plus Minus is okay, same sign is only in some region well handled

In [18]:
#for comparision
for x in startX:
    reached = "     failed     "
    x_reach, lamb_reach = optimise(x, -1, gradf, hessianf, h, gradh, hessianh, steps)
    if np.linalg.norm(x_reach-x_optimal) < 1e-5:
        reached = "'''successful'''"
        if np.linalg.norm(lamb_reach-lamb_optimal) < 1e-4:
            reached = "|||super successful|||"
    print(f"{reached}.  after {steps} iterations we got from {x} with initial lambda {lamb} to {x_reach, lamb_reach}.  Distance:{np.linalg.norm(x_reach-x_optimal)}")

|||super successful|||.  after 500 iterations we got from [ 0.90927292 96.70620254] with initial lambda 10 to (array([-0.74833549,  0.66332043]), array([-0.21232494])).  Distance:4.534001547404924e-06
     failed     .  after 500 iterations we got from [  7.9003086  -14.26412433] with initial lambda 10 to (array([ 0.01434523, -0.9998971 ]), array([109.16259756])).  Distance:1.829748586656503
     failed     .  after 500 iterations we got from [83.89993268 49.98909658] with initial lambda 10 to (array([2653.69659182, -120.03881577]), array([-7.48038694e+156])).  Distance:2657.1877806486614
|||super successful|||.  after 500 iterations we got from [-1.77267454 19.47635788] with initial lambda 10 to (array([-0.74833549,  0.66332043]), array([-0.21232494])).  Distance:4.534001547515435e-06
|||super successful|||.  after 500 iterations we got from [-35.66594606  70.77626538] with initial lambda 10 to (array([-0.74833549,  0.66332043]), array([-0.21232494])).  Distance:4.534001547515435e-06


With a constant lambda we can see, that espacially for the randomly generated points the Newton like method performs poorly as expected.

#### Ex3
Merit function
$\\$
$M(x_1,x_2) = f(x_1,x_2)+ρh(x_1,x_2)^2$

In [29]:
def grd_M(x):
    g= np.array([3*math.e**(3*x[0])+40*x[0]**3+40*x[0]*x[1]**2-40*x[0], -4*math.e**(-4*x[1])+40*x[1]**3+40*x[1]*x[0]**2-40*x[1]])
    return g

def grad_desc(x,a=0.01):
    g = grd_M(x)
    normed = g/np.linalg.norm(g)
    return x-a*normed


def meritOptimise(start, grd_M, steps, step=0.01):
    x = start
    for c in range(steps):
        x = grad_desc(x, step)
        if np.linalg.norm(grd_M(x)) < 1e-2:
            print(f"exited after {c} iterations with gradient {grd_M(x)}")
            break

    return x

In [30]:

# test
for x in startX:
    reached = "     failed     "
    x_reach = meritOptimise(x, grd_M, 50000)
    if np.linalg.norm(x_reach-x_optimal) < 5e-2:
        reached = "'''successful'''"
        if np.linalg.norm(x_reach-x_optimal) < 1e-3:
            reached = "|||super successful|||"
    print(f"{reached}.  We got from {x} to {x_reach}. Distance:{np.linalg.norm(x_reach-x_optimal)}")

'''successful'''.  We got from [ 0.90927292 96.70620254] to [-0.75028816  0.66464703]. Distance:0.0023571885458530443
'''successful'''.  We got from [  7.9003086  -14.26412433] to [-0.75062321  0.66494398]. Distance:0.0028018496067556274
'''successful'''.  We got from [83.89993268 49.98909658] to [-0.75398634  0.66792919]. Distance:0.00728874698359742
'''successful'''.  We got from [-1.77267454 19.47635788] to [-0.7466029   0.66137461]. Distance:0.002608075809316768
|||super successful|||.  We got from [-35.66594606  70.77626538] to [-0.74866171  0.66320421]. Distance:0.0003419152253067709
|||super successful|||.  We got from [-48.06524546  34.73730522] to [-0.74865349  0.66319691]. Distance:0.00033678909774926446
'''successful'''.  We got from [87.27047721  1.40994007] to [-0.7502794   0.66463926]. Distance:0.002345575543294348
'''successful'''.  We got from [-92.78055928   6.53848317] to [-0.75314571  0.6671811 ]. Distance:0.006164649284228727
'''successful'''.  We got from [-19.3265

As we can see, we are getting pretty close on a lot(more then the Newton like Method before) of solutions and on some of them even pretty close. But in general not as close as with the Newton Method. So the Merit Optimiser can be used to get an approximate result but not the most correct one, although it works in many more cases than the newton method.

#### Ex4
Let's combine both methods and use the strength of both in order to achieve the best possible results

In [24]:
def combinedMethod(x_start, lamb_start, rho, gradf, hessianf, h, gradh, hessianh, steps_Newton=100, steps_Merit=5000, stepsize=0.1):
    grd_M = lambda x: gradf(x) + 2*rho*gradh(x)
    x_merit = meritOptimise(x_start, grd_M, steps_Merit, stepsize)

    x_end = optimise(x_merit, lamb_start, gradf, hessianf, h, gradh, hessianh, steps_Newton)
    return x_merit, x_end

#test how far can we go
x_1, x_2 = combinedMethod(np.array([-50, 100]), 5, 10, gradf, hessianf, h, gradh, hessianh)
print(x_1, x_2)

[-0.70205996  0.62232015] (array([-0.74833549,  0.66332043]), array([-0.21232494]))


In order to get the best results we decided to first use the merit optimiser so that we will get a good enough starting point and then use the newton descent method to get us the best possible result. We decided to use 5000 maximum steps for the Merit Optimiser and then just 100 steps for the Newton Optimiser.

$(-0.70205996 , 0.62232015)$  are the results from the Merit Optimiser. We can see that it's very close to the wanted result. This is a very good starting point for the newton method. In the end, by using both methods we could achieve the expected results of   $(-0.74833549,  0.66332043)$  .

In [21]:
#test on all starting points
for x in startX:
    reached = "     failed     "
    x_merit, [x_reach, lamb_reach] = combinedMethod(x, 1, 10, gradf, hessianf, h, gradh, hessianh)
    print(x_reach)
    if np.linalg.norm(x_reach-x_optimal) < 5e-5:
        reached = "'''successful'''"
        if np.linalg.norm(x_reach-x_optimal) < 1e-5:
            reached = "|||super successful|||"
    print(f"{reached}.  We got from {x} to {x_reach}.  Distance:{np.linalg.norm(x_reach-x_optimal)}")


[-0.74833549  0.66332043]
|||super successful|||.  We got from [ 0.90927292 96.70620254] to [-0.74833549  0.66332043].  Distance:4.53400154739428e-06
[-0.74833549  0.66332043]
|||super successful|||.  We got from [  7.9003086  -14.26412433] to [-0.74833549  0.66332043].  Distance:4.534001547515435e-06
[-0.74833549  0.66332043]
|||super successful|||.  We got from [83.89993268 49.98909658] to [-0.74833549  0.66332043].  Distance:4.53400154739428e-06
[-0.74833549  0.66332043]
|||super successful|||.  We got from [-1.77267454 19.47635788] to [-0.74833549  0.66332043].  Distance:4.53400154739428e-06
[-0.74833549  0.66332043]
|||super successful|||.  We got from [-35.66594606  70.77626538] to [-0.74833549  0.66332043].  Distance:4.53400154739428e-06
[-0.74833549  0.66332043]
|||super successful|||.  We got from [-48.06524546  34.73730522] to [-0.74833549  0.66332043].  Distance:4.534001547404924e-06
[-0.74833549  0.66332043]
|||super successful|||.  We got from [87.27047721  1.40994007] to 

As expected combining both methods leads to even better results, because we reach the proximity of the solution by minimizing the merit function and can converge to the optimal solution very fast afterwards by using the Newton-like Method.
But there are some points which neither of the methods can't handle, for example $x_0=(100, -100)$
